<a href="https://colab.research.google.com/github/jjikku/OptiGene-Algo/blob/main/ICT_Group_2_ML_AI_Algo_Trading_data_aggregation_pre_processing_weekly_9_16_Sep_2021_test_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Specify the exact path to the CSV file
file_path = "/content/drive/My Drive/NIfty_Dataset/09-16-SEP-2021-weekly-expiry_data__VEGE_NF_AND_BNF_Options.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Exclude rows where the ticker contains any of the following
excluded_tickers = [
    "BANKNIFTY", "FINNIFTY", "FUT", "NIFTYAUTO", "NIFTYENERGY",
    "NIFTYFINSERVICE", "NIFTYFMCG", "NIFTYINFRA", "NIFTYIT", "NIFTYMEDIA",
    "NIFTYMETAL", "NIFTYMIDCAP50", "NIFTYPHARMA", "NIFTYPSUBANK",
    "NIFTYREALTY", "USDINR-I"
]

# Create a regular expression pattern that matches any of the excluded tickers
pattern = '|'.join(excluded_tickers)

# Apply the mask to filter the DataFrame
mask = ~df['Ticker'].str.contains(pattern)
df_filtered = df[mask]

# Get unique values in the 'Ticker' column after filtering
unique_tickers = df_filtered['Ticker'].unique()

# Extract all rows that match the unique tickers
final_df = df[df['Ticker'].isin(unique_tickers)]

# Create a new CSV file with the extracted rows
output_file_path = "/content/drive/My Drive/NIfty_Dataset/weekly_filtered_tickers_data_9_16_Sep.csv"
final_df.to_csv(output_file_path, index=False)

print(f"Created new CSV file at: {output_file_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Created new CSV file at: /content/drive/My Drive/NIfty_Dataset/weekly_filtered_tickers_data_9_16_Sep.csv


In [ ]:
# print(unique_tickers)


In [ ]:
# final_df.count()


In [ ]:
# final_df['Open Interest'].count()

In [ ]:
# non_zero_count = final_df['Open Interest'].ne(0).sum()
# print(f"The count of non-zero 'Open Interest' values is: {non_zero_count}")


In [ ]:

# Create a separate DataFrame for each unique ticker and store it in a dictionary
dfs = {ticker: df[df['Ticker'] == ticker] for ticker in unique_tickers}

# Now dfs is a dictionary with tickers as keys and the corresponding DataFrame as values

# Assuming df is your original DataFrame with all tickers
for ticker in unique_tickers:
    # Select the DataFrame for the current ticker
    ticker_df = df[df['Ticker'] == ticker].copy()  # Use copy to avoid SettingWithCopyWarning

    # Convert 'Date/Time' to datetime and set as index
    ticker_df['Date/Time'] = pd.to_datetime(ticker_df['Date/Time'], format='%d-%m-%Y %H:%M:%S')
    ticker_df.set_index('Date/Time', inplace=True)

    # Remove duplicate indices if any
    ticker_df = ticker_df[~ticker_df.index.duplicated(keep='first')]

    # Assign the processed DataFrame back to the dfs dictionary
    dfs[ticker] = ticker_df

# Now dfs is a dictionary where each value is a DataFrame with 'Date/Time' as a datetime index


In [ ]:
# Iterate through the dfs dictionary
for ticker, dataframe in dfs.items():
    # Check if the ticker ends with 'PE' or 'CE'
    if ticker.endswith('PE') or ticker.endswith('CE'):
        # Forward fill any NaN values
        dataframe['Open Interest'] = dataframe['Open Interest'].fillna(method='ffill')

        # Replace zero values with the forward filled values
        dataframe['Open Interest'] = dataframe['Open Interest'].replace(to_replace=0, method='ffill')

        # Assign the processed DataFrame back to the dfs dictionary
        dfs[ticker] = dataframe

In [ ]:
import numpy as np
# Get the DataFrame for NIFTY as the reference
reference_df = dfs['NIFTY'].copy()

# Get all timestamps from the NIFTY DataFrame
reference_timestamps = reference_df.index
print("REF",reference_timestamps)

# Function to insert missing timestamps into a DataFrame
def insert_missing_timestamps(df, reference_ts):
    # Ensure the DataFrame index is unique and sorted
    df = df[~df.index.duplicated(keep='first')].sort_index()

    # Find the missing timestamps
    missing_timestamps = reference_ts.difference(df.index)

    # Insert rows with NaNs for missing timestamps
    for ts in missing_timestamps:
        if ts not in df.index:
            df.loc[ts] = [np.nan] * len(df.columns)

    # Sort the DataFrame after inserting new rows
    df.sort_index(inplace=True)

    return df

# Iterate over each DataFrame in dfs
for ticker, dataframe in dfs.items():
    if ticker != 'NIFTY':  # Skip the NIFTY DataFrame as it is the reference
        # Insert missing timestamps without altering existing data
        print("TICKER",ticker)
        dfs[ticker] = insert_missing_timestamps(dataframe, reference_timestamps)


REF DatetimeIndex(['2021-09-03 09:15:00', '2021-09-03 09:16:00',
               '2021-09-03 09:17:00', '2021-09-03 09:18:00',
               '2021-09-03 09:19:00', '2021-09-03 09:20:00',
               '2021-09-03 09:21:00', '2021-09-03 09:22:00',
               '2021-09-03 09:23:00', '2021-09-03 09:24:00',
               ...
               '2021-09-16 15:21:00', '2021-09-16 15:22:00',
               '2021-09-16 15:23:00', '2021-09-16 15:24:00',
               '2021-09-16 15:25:00', '2021-09-16 15:26:00',
               '2021-09-16 15:27:00', '2021-09-16 15:28:00',
               '2021-09-16 15:29:00', '2021-09-16 15:30:00'],
              dtype='datetime64[ns]', name='Date/Time', length=3384, freq=None)
TICKER INDIAVIX
TICKER NIFTYWK16000CE
TICKER NIFTYWK16000PE
TICKER NIFTYWK16050CE
TICKER NIFTYWK16050PE
TICKER NIFTYWK16100CE
TICKER NIFTYWK16100PE
TICKER NIFTYWK16150CE
TICKER NIFTYWK16150PE
TICKER NIFTYWK16200CE
TICKER NIFTYWK16200PE
TICKER NIFTYWK16250CE
TICKER NIFTYWK16250PE
TICKER

In [ ]:
# # Define trading hours
# trading_hours = pd.date_range(start='09:15:00', end='15:30:00', freq='1min')

# # Get the DataFrame for NIFTY as the reference
# reference_df = dfs['NIFTY'].copy()

# # Generate a full range of timestamps for all trading days in the reference NIFTY DataFrame
# all_trading_days = reference_df.index.normalize().unique()
# all_possible_timestamps = pd.DatetimeIndex([pd.Timestamp(day) + pd.Timedelta(minutes=m.minute + m.hour * 60)
#                                             for day in all_trading_days for m in trading_hours])

# # Now update all DataFrames in dfs to have the same timestamps as the reference NIFTY DataFrame
# for ticker, dataframe in dfs.items():
#     # Reindex to the full range of possible timestamps, forward filling the missing data
#     dataframe = dataframe.reindex(all_possible_timestamps, method='ffill')

#     # Forward fill the Open Interest, if the ticker ends with 'PE' or 'CE'
#     if ticker.endswith(('PE', 'CE')):
#         dataframe['Open Interest'].fillna(method='ffill', inplace=True)
#         dataframe['Open Interest'].replace(to_replace=0, method='ffill', inplace=True)

#     # Update the dictionary with the modified DataFrame
#     dfs[ticker] = dataframe

# # After this loop, each DataFrame in dfs should have no missing timestamps and the 'Open Interest' column should be forward filled.


In [ ]:
# Define the market open and close times
market_open_time = '09:15:00'
market_close_time = '15:30:00'

# Create a new dictionary to store the filled DataFrames
filled_dfs = {}

for ticker, dataframe in dfs.items():
    # Copy the DataFrame to avoid modifying the original
    df_filled = dataframe.copy()

    # Get unique trading days
    trading_days = df_filled.index.normalize().unique()

    # Forward fill for each day
    for day in trading_days:
        # Select the data for the current trading day
        day_data = df_filled[df_filled.index.date == day.date()]

        # Forward fill the data for the current day
        df_filled.loc[day_data.index] = day_data.fillna(method='ffill')

    # Now, backward fill the entire DataFrame to cover the start of the day
    df_filled = df_filled.fillna(method='bfill')

    # Assign the filled DataFrame to the new dictionary
    filled_dfs[ticker] = df_filled


In [ ]:

# Assuming filled_dfs is your dictionary containing DataFrames with filled values
for ticker in unique_tickers:
    # Select the DataFrame for the current ticker
    ticker_df = filled_dfs[ticker].copy()  # Use copy to avoid SettingWithCopyWarning

    # Since filled_dfs already contains the DataFrames with 'Date/Time' as a datetime index,
    #  do not need to convert 'Date/Time' again, and  can use ticker_df directly if needed

    # Remove duplicate indices if any (if this step is still necessary)
    ticker_df = ticker_df[~ticker_df.index.duplicated(keep='first')]


    # Assign the processed DataFrame back to the dfs dictionary
    dfs[ticker] = ticker_df

# Now dfs is a dictionary where each value is a DataFrame with 'Date/Time' as a datetime index


In [ ]:
print(dfs['NIFTY'])

                    Ticker     Open     High      Low    Close  Volume  \
Date/Time                                                                
2021-09-03 09:15:00  NIFTY  17262.5  17296.8  17255.3  17293.3       0   
2021-09-03 09:16:00  NIFTY  17292.3  17300.3  17289.0  17289.0       0   
2021-09-03 09:17:00  NIFTY  17288.9  17298.2  17282.0  17297.3       0   
2021-09-03 09:18:00  NIFTY  17298.8  17312.0  17296.5  17302.3       0   
2021-09-03 09:19:00  NIFTY  17302.3  17306.7  17301.2  17306.1       0   
...                    ...      ...      ...      ...      ...     ...   
2021-09-16 15:26:00  NIFTY  17632.3  17635.4  17631.3  17633.0       0   
2021-09-16 15:27:00  NIFTY  17635.3  17635.4  17630.3  17631.8       0   
2021-09-16 15:28:00  NIFTY  17629.5  17630.7  17626.3  17630.2       0   
2021-09-16 15:29:00  NIFTY  17630.6  17630.6  17624.5  17627.5       0   
2021-09-16 15:30:00  NIFTY  17626.5  17626.5  17626.5  17626.5       0   

                     Open Interest  


In [ ]:
# Assuming dfs['NIFTY'] is the DataFrame you want to save
nifty_df = dfs['NIFTY']

# Specify the path in your Google Drive where you want to save the CSV
csv_file_path = '/content/drive/My Drive/NIfty_Dataset/NIFTY_9_16_Sep_2021.csv'

# Save the DataFrame to CSV
nifty_df.to_csv(csv_file_path, index=True)

print(f"DataFrame saved to {csv_file_path}")


DataFrame saved to /content/drive/My Drive/NIfty_Dataset/NIFTY_9_16_Sep_2021.csv


In [ ]:
# Assuming dfs['NIFTY'] is the DataFrame you want to save
indiavix_df = dfs['INDIAVIX']

# Specify the path in your Google Drive where you want to save the CSV
csv_file_path = '/content/drive/My Drive/NIfty_Dataset/INDIAVIX_9_16_Sep_2021.csv'

# Save the DataFrame to CSV
indiavix_df.to_csv(csv_file_path, index=True)

print(f"DataFrame saved to {csv_file_path}")

DataFrame saved to /content/drive/My Drive/NIfty_Dataset/INDIAVIX_9_16_Sep_2021.csv


In [ ]:
!pip install mibian


In [ ]:
indiavix_df = pd.read_csv('/content/drive/My Drive/NIfty_Dataset/INDIAVIX_9_16_Sep_2021.csv')
# Print the columns to check the exact name of the 'Date/Time' column
print(indiavix_df.columns)

Index(['Date/Time', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Open Interest'],
      dtype='object')


In [ ]:
print(indiavix_df.index)


RangeIndex(start=0, stop=3384, step=1)


In [ ]:
!pip install datetime

In [ ]:
import pandas as pd
import numpy as np
import mibian
from datetime import time, datetime, timedelta


indiavix_df = pd.read_csv('/content/drive/My Drive/NIfty_Dataset/INDIAVIX_9_16_Sep_2021.csv', index_col='Date/Time', parse_dates=['Date/Time'])
indiavix_df.rename(columns={'Close': 'INDIAVIX'}, inplace=True)

# Load the NIFTY data
nifty_df = pd.read_csv('/content/drive/My Drive/NIfty_Dataset/NIFTY_9_16_Sep_2021.csv', index_col='Date/Time', parse_dates=['Date/Time'])

# Join INDIAVIX data with NIFTY data
nifty_df = nifty_df.join(indiavix_df['INDIAVIX'], how='left')
nifty_df['INDIAVIX'].fillna(method='ffill', inplace=True)  # Fill NaN values with the forward fill method


# Check if the 'INDIAVIX' column exists after the join operation
if 'INDIAVIX' not in nifty_df.columns:
    raise KeyError('The INDIAVIX column was not found after the join operation. Please check the column names and join operation.')

print("NIFTYDF",nifty_df)
print("columns",nifty_df.columns)

# Constants
interest_rate = 3.32  # 3-month Indian government bond yield as annual percentage
days_in_year = 365  # Days in a year for the Greeks' calculations
strike_step = 50  # Known step between strike prices
minutes_in_a_year = 365 * 24 * 60  # Total minutes in a year
market_close_time = time(15, 30)



def calculate_minutes_to_expiry(current_timestamp, expiry_date, market_close_time):
    # Ensure market_close_time is a datetime.time object
    if isinstance(market_close_time, pd.Timestamp):
        market_close_time = market_close_time.time()

    closing_time = datetime.combine(expiry_date, market_close_time)

    # If the current timestamp is on the expiry date
    if current_timestamp.date() == expiry_date.date():
        # Calculate the remaining time until market close
        return max(int((closing_time - current_timestamp).total_seconds() / 60), 0)
    else:
        # Calculate time from the current timestamp to the end of day
        end_of_day = datetime.combine(current_timestamp.date(), time(23, 59))  # Last minute of the current day
        minutes_today = int((end_of_day - current_timestamp).total_seconds() / 60) + 1  # Plus one for the last minute

        # Calculate full days until the expiry date
        full_days = (expiry_date.date() - current_timestamp.date()).days - 1  # Minus one because we already counted today

        # Calculate minutes from the start of the expiry day to the market close
        start_of_expiry_day = datetime.combine(expiry_date.date(), time(0, 0))  # Midnight of the expiry day
        minutes_expiry_day = int((closing_time - start_of_expiry_day).total_seconds() / 60)

        # Total minutes is the sum of all parts
        return minutes_today + (full_days * 24 * 60) + minutes_expiry_day


# Function to calculate the Greeks using the Black-Scholes model
def calculate_greeks(underlying_price, strike_price, minutes_to_expiry, volatility, interest_rate, option_type):
    # Convert minutes to expiry into years for the Black-Scholes model
    time_to_expiry_in_years = minutes_to_expiry / minutes_in_a_year

    # Ensure we do not have a division by zero or negative time to expiry
    if time_to_expiry_in_years <= 0:
        return {greek: 0 for greek in ['Delta', 'Gamma', 'Theta', 'Vega', 'Rho']}

    # Convert annual interest rate and volatility from percentages to decimals
    annual_interest_rate = interest_rate / 100
    annual_volatility = volatility / 100

    # Black-Scholes model calculations
    bs = mibian.BS([underlying_price, strike_price, annual_interest_rate * 100, time_to_expiry_in_years * 365], volatility=annual_volatility * 100)

    # Return a dictionary of Greeks
    return {
        'Delta': bs.callDelta if option_type == 'call' else bs.putDelta,
        'Gamma': bs.gamma,
        'Theta': bs.callTheta / 365 if option_type == 'call' else bs.putTheta / 365,
        'Vega': bs.vega / 100,  # per 1% change in IV
        'Rho': bs.callRho / 100 if option_type == 'call' else bs.putRho / 100  # per 1% change in interest rate
    }


# Function to populate the DataFrame with strikes and Greeks
def populate_greeks(nifty_df, indiavix_df, strike_step, dfs):

    expiry_date = nifty_df.index.max().normalize() + pd.Timedelta(hours=15, minutes=30)
    print("EXPIRY DAY", expiry_date)
    market_close_time = expiry_date #pd.to_datetime(expiry_date.strftime('%Y-%m-%d') + ' 15:30:00')  # Assuming 3:30 PM market close

    # Create columns for Greeks in the dataframe
    for strike_type in ['ATM', 'ITM1', 'OTM1', 'ITM2', 'OTM2', 'ITM3', 'OTM3', 'ITM4', 'OTM4', 'ITM5', 'OTM5']:
        for greek in ['Delta', 'Gamma', 'Theta', 'Vega', 'Rho']:
            nifty_df[f'{strike_type}_{greek}'] = np.nan
            nifty_df[f'{strike_type}_OI'] = np.nan
            nifty_df[f'{strike_type}_Call_Volume'] = np.nan
            nifty_df[f'{strike_type}_Put_Volume'] = np.nan

    nifty_df['PCR'] = np.nan

    # Iterate over the dataframe and calculate Greeks
    for timestamp, row in nifty_df.iterrows():
        total_call_oi = 0
        total_put_oi = 0
        nifty_close = row['Close']
        # Find ATM, ITM, and OTM strikes
        atm_strike = int(round(nifty_close / strike_step) * strike_step)  # Closest strike
        itm_strikes = [atm_strike - i * strike_step for i in range(1, 6)]
        otm_strikes = [atm_strike + i * strike_step for i in range(1, 6)]
        all_strikes = [atm_strike] + itm_strikes + otm_strikes

        print(f"Processing timestamp: {timestamp}")

        # Get VIX value for the timestamp
        vix_value = indiavix_df.loc[timestamp, 'INDIAVIX'] if timestamp in indiavix_df.index else np.nan
        if vix_value is None:
            print(f"VIX value is missing for timestamp: {timestamp}")
            continue  # Skip this iteration if VIX value is missing


        minutes_to_expiry = calculate_minutes_to_expiry(timestamp, expiry_date, market_close_time)
        print("Current time & days to expiry:", timestamp.date(), minutes_to_expiry / (60 * 24))

        print("Curr time & days to EXpiry", timestamp.date(),minutes_to_expiry/(60*24))
        # Calculate Greeks for all strikes
        for i, strike in enumerate(all_strikes, start=1):
            if i == 1:  # ATM
                strike_type = 'ATM'
            elif i <= 6:  # ITM strikes
                strike_type = f'ITM{i-1}'
            else:  # OTM strikes
                strike_type = f'OTM{i-6}'

            call_key = f'NIFTYWK{strike}CE'
            put_key = f'NIFTYWK{strike}PE'

            call_volume = dfs[call_key].loc[timestamp, 'Volume'] if timestamp in dfs[call_key].index else np.nan
            put_volume = dfs[put_key].loc[timestamp, 'Volume'] if timestamp in dfs[put_key].index else np.nan

            # Populate volume data
            nifty_df.at[timestamp, f'{strike_type}_Call_Volume'] = call_volume
            nifty_df.at[timestamp, f'{strike_type}_Put_Volume'] = put_volume

            # print(f"Retrieving data for strike {strike} ({call_key} and {put_key})")

             # Check if the keys are present in the dfs dictionary
            if call_key not in dfs or put_key not in dfs:
                print(f"Missing data for strike {strike}: {call_key} or {put_key} not in dfs")
                continue

            # Check if the timestamp exists in the dfs DataFrames
            if timestamp not in dfs[call_key].index:
                print(f"Timestamp {timestamp} not found in dfs for {call_key}")
                continue
            if timestamp not in dfs[put_key].index:
                print(f"Timestamp {timestamp} not found in dfs for {put_key}")
                continue

            call_close = dfs[call_key].loc[timestamp, 'Close'] if timestamp in dfs[call_key].index else np.nan
            put_close = dfs[put_key].loc[timestamp, 'Close'] if timestamp in dfs[put_key].index else np.nan
            call_oi = dfs[call_key].loc[timestamp, 'Open Interest'] if timestamp in dfs[call_key].index else np.nan
            put_oi = dfs[put_key].loc[timestamp, 'Open Interest'] if timestamp in dfs[put_key].index else np.nan
            # print(f"Call Close: {call_close}, Put Close: {put_close}, Call OI: {call_oi}, Put OI: {put_oi}")

            #print("STRIKE",nifty_close, strike)
            # Calculate Greeks using the close prices
            if not np.isnan(call_close):
                greeks = calculate_greeks(nifty_close, strike, minutes_to_expiry, call_close, interest_rate, option_type='call')
                # Populate Greeks for call
                for greek_name, greek_value in greeks.items():
                    nifty_df.at[timestamp, f'{strike_type}_{greek_name}'] = greek_value
                    nifty_df.at[timestamp, f'{strike_type}_OI'] = call_oi  # Populate OI for call

            if not np.isnan(put_close):
                greeks = calculate_greeks(nifty_close, strike, minutes_to_expiry, put_close, interest_rate, option_type='put')
                # Populate Greeks for put
                for greek_name, greek_value in greeks.items():
                    nifty_df.at[timestamp, f'{strike_type}_PE_{greek_name}'] = greek_value
                    nifty_df.at[timestamp, f'{strike_type}_PE_OI'] = put_oi  # Populate OI for put

    return nifty_df

# Populate the DataFrame with Greeks
nifty_with_greeks = populate_greeks(nifty_df, indiavix_df, strike_step, dfs)

def calculate_pcr(row):
    total_call_oi = sum(row.get(f'{strike}_OI', 0) for strike in ['ATM'] + [f'ITM{i}' for i in range(1, 6)] + [f'OTM{i}' for i in range(1, 6)])
    total_put_oi = sum(row.get(f'{strike}_PE_OI', 0) for strike in ['ATM'] + [f'ITM{i}' for i in range(1, 6)] + [f'OTM{i}' for i in range(1, 6)])
    print("OI", total_call_oi, total_put_oi)
    return total_put_oi / total_call_oi if total_call_oi != 0 else np.nan

# Function to calculate PCR for volume
def calculate_pcr_volume(row):
    total_call_volume = sum(row.get(f'{strike}_Call_Volume', 0) for strike in ['ATM'] + [f'ITM{i}' for i in range(1, 6)] + [f'OTM{i}' for i in range(1, 6)])
    total_put_volume = sum(row.get(f'{strike}_Put_Volume', 0) for strike in ['ATM'] + [f'ITM{i}' for i in range(1, 6)] + [f'OTM{i}' for i in range(1, 6)])
    return total_put_volume / total_call_volume if total_call_volume != 0 else np.nan

# Apply PCR calculation to each row
nifty_with_greeks['PCR'] = nifty_with_greeks.apply(calculate_pcr, axis=1)
print(nifty_with_greeks.head())
# Apply PCR calculation for volume to each row
nifty_with_greeks['PCR_Volume'] = nifty_with_greeks.apply(calculate_pcr_volume, axis=1)
# # Output the DataFrame to a CSV
nifty_with_greeks.to_csv('/content/drive/My Drive/NIfty_Dataset/NIFTY_with_Greeks_9_16_Sep_2021.csv')

print("Greeks and Open Interest data have been populated and the DataFrame has been saved to CSV.")


Streaming output truncated to the last 5000 lines.
Current time & days to expiry: 2021-09-15 1.1027777777777779
Curr time & days to EXpiry 2021-09-15 1.1027777777777779
Processing timestamp: 2021-09-15 13:03:00
Current time & days to expiry: 2021-09-15 1.1020833333333333
Curr time & days to EXpiry 2021-09-15 1.1020833333333333
Processing timestamp: 2021-09-15 13:04:00
Current time & days to expiry: 2021-09-15 1.101388888888889
Curr time & days to EXpiry 2021-09-15 1.101388888888889
Processing timestamp: 2021-09-15 13:05:00
Current time & days to expiry: 2021-09-15 1.1006944444444444
Curr time & days to EXpiry 2021-09-15 1.1006944444444444
Processing timestamp: 2021-09-15 13:06:00
Current time & days to expiry: 2021-09-15 1.1
Curr time & days to EXpiry 2021-09-15 1.1
Processing timestamp: 2021-09-15 13:07:00
Current time & days to expiry: 2021-09-15 1.0993055555555555
Curr time & days to EXpiry 2021-09-15 1.0993055555555555
Processing timestamp: 2021-09-15 13:08:00
Current time & days t

In [ ]:
print(nifty_with_greeks.head())


                    Ticker     Open     High      Low    Close  Volume  \
Date/Time                                                                
2021-09-03 09:15:00  NIFTY  17262.5  17296.8  17255.3  17293.3       0   
2021-09-03 09:16:00  NIFTY  17292.3  17300.3  17289.0  17289.0       0   
2021-09-03 09:17:00  NIFTY  17288.9  17298.2  17282.0  17297.3       0   
2021-09-03 09:18:00  NIFTY  17298.8  17312.0  17296.5  17302.3       0   
2021-09-03 09:19:00  NIFTY  17302.3  17306.7  17301.2  17306.1       0   

                     Open Interest  INDIAVIX  ATM_Delta     ATM_OI  ...  \
Date/Time                                                           ...   
2021-09-03 09:15:00              0     14.19   0.542985  1666100.0  ...   
2021-09-03 09:16:00              0     14.21   0.541113  1666100.0  ...   
2021-09-03 09:17:00              0     14.17   0.543083  2063500.0  ...   
2021-09-03 09:18:00              0     14.14   0.544595  2063500.0  ...   
2021-09-03 09:19:00            

In [ ]:
print("PCR_Volume" in nifty_with_greeks.columns)

True


In [ ]:
pd.set_option('display.max_columns', None)
print(nifty_with_greeks.columns.tolist())


['Ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'Open Interest', 'INDIAVIX', 'ATM_Delta', 'ATM_OI', 'ATM_Call_Volume', 'ATM_Put_Volume', 'ATM_Gamma', 'ATM_Theta', 'ATM_Vega', 'ATM_Rho', 'ITM1_Delta', 'ITM1_OI', 'ITM1_Call_Volume', 'ITM1_Put_Volume', 'ITM1_Gamma', 'ITM1_Theta', 'ITM1_Vega', 'ITM1_Rho', 'OTM1_Delta', 'OTM1_OI', 'OTM1_Call_Volume', 'OTM1_Put_Volume', 'OTM1_Gamma', 'OTM1_Theta', 'OTM1_Vega', 'OTM1_Rho', 'ITM2_Delta', 'ITM2_OI', 'ITM2_Call_Volume', 'ITM2_Put_Volume', 'ITM2_Gamma', 'ITM2_Theta', 'ITM2_Vega', 'ITM2_Rho', 'OTM2_Delta', 'OTM2_OI', 'OTM2_Call_Volume', 'OTM2_Put_Volume', 'OTM2_Gamma', 'OTM2_Theta', 'OTM2_Vega', 'OTM2_Rho', 'ITM3_Delta', 'ITM3_OI', 'ITM3_Call_Volume', 'ITM3_Put_Volume', 'ITM3_Gamma', 'ITM3_Theta', 'ITM3_Vega', 'ITM3_Rho', 'OTM3_Delta', 'OTM3_OI', 'OTM3_Call_Volume', 'OTM3_Put_Volume', 'OTM3_Gamma', 'OTM3_Theta', 'OTM3_Vega', 'OTM3_Rho', 'ITM4_Delta', 'ITM4_OI', 'ITM4_Call_Volume', 'ITM4_Put_Volume', 'ITM4_Gamma', 'ITM4_Theta', 'ITM4_Vega', 'I

In [ ]:
print(nifty_with_greeks['PCR_Volume'])

Date/Time
2021-09-03 09:15:00    0.400507
2021-09-03 09:16:00    0.475421
2021-09-03 09:17:00    0.644477
2021-09-03 09:18:00    0.492156
2021-09-03 09:19:00    0.564910
                         ...   
2021-09-16 15:26:00    2.357391
2021-09-16 15:27:00    1.685187
2021-09-16 15:28:00    1.463176
2021-09-16 15:29:00    1.982279
2021-09-16 15:30:00    1.982279
Name: PCR_Volume, Length: 3384, dtype: float64
